In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import activations
from tensorflow.keras.layers import (Conv2D, Dense, Concatenate, Add, ReLU, Input, GlobalAveragePooling2D, Activation,
                                     Conv1D, Multiply, Permute)
from tensorflow.keras import optimizers
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from matplotlib import pyplot as plt
from keras.callbacks import CSVLogger
from collections import deque
import gc
import copy


config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)


root = os.getcwd()



In [2]:
path_post_enc = "C:\\Users\\Furkan\\Desktop\\BLG4901E\\nii_deney\\post_enc_frames"
path_pre_enc = "C:\\Users\\Furkan\\Desktop\\BLG4901E\\nii_deney\\pre_enc_frames"
batch_size = 10


def data_generator(start_index, batch_size=batch_size):
    X = []
    y = []
    for i in range(start_index, start_index + batch_size):
        folders = os.listdir(path_post_enc)
        os.chdir(os.path.join(path_post_enc, folders[i]))
        files = os.listdir(os.getcwd())
        for file in files:
            with open(file, 'rb') as yuv_file:
                X.append(np.frombuffer(yuv_file.read(256 * 256), dtype=np.uint8).reshape((256, 256, 1)))

    for i in range(start_index, start_index + batch_size):
        folders = os.listdir(path_pre_enc)
        os.chdir(os.path.join(path_pre_enc, folders[i]))
        files = os.listdir(os.getcwd())
        for file in files:
            with open(file, 'rb') as yuv_file:
                y.append(np.frombuffer(yuv_file.read(256 * 256), dtype=np.uint8).reshape((256, 256, 1)))

    X = np.array(X) / 255.0
    y = np.array(y) / 255.0

    return X, y, start_index + batch_size

In [3]:
def dc_block(input):
    x = Conv2D(filters=8, kernel_size=3, strides=1, padding='same')(input)

    x1 = Dense(12, activation='relu')(x)
    x1 = Conv2D(filters=12, kernel_size=3, strides=1,
                padding='same', activation='relu')(x1)

    x2 = Dense(16, activation='relu')(x1)
    x2 = Dense(16, activation='relu')(x2)
    x2 = Conv2D(filters=16, kernel_size=3, strides=1,
                padding='same', activation='relu')(x2)

    x3 = Dense(20, activation='relu')(x2)
    x3 = Dense(20, activation='relu')(x3)
    x3 = Dense(20, activation='relu')(x3)
    x3 = Conv2D(filters=20, kernel_size=3, strides=1,
                padding='same', activation='relu')(x3)

    x4 = Dense(24, activation='relu')(x3)
    x4 = Dense(24, activation='relu')(x4)
    x4 = Dense(24, activation='relu')(x4)
    x4 = Dense(24, activation='relu')(x4)
    x4 = Conv2D(filters=24, kernel_size=3, strides=1,
                padding='same', activation='relu')(x4)

    x5 = Dense(24, activation='relu')(x4)
    x5 = Concatenate()([Dense(8)(input), x5])
    x5 = Conv2D(filters=16, kernel_size=3, strides=1,
                padding='same', activation='relu')(x5)

    x6 = Add()([x5, input])
    out = Conv2D(filters=16, kernel_size=3, strides=1,
                 padding='same', activation='relu')(x6)

    return out


def dcr_net(input_shape=(None, None, 1), filters=16,
            name='DCRN_model'):  # ORIGINAL 64 filters!!!
    lr_image = Input(shape=input_shape, name='input')
    x_start = Conv2D(filters, kernel_size=3, strides=1,
                     padding='same')(lr_image)
    x_start = ReLU()(x_start)
    x = dc_block(x_start)
    x1 = dc_block(x)
    x2 = dc_block(x1)
    x3 = Concatenate()([Dense(16)(x_start), x, x1, x2])
    x4 = Conv2D(filters, kernel_size=3, strides=1, padding='same')(x3)
    x4 = ReLU()(x4)
    # x5 = cab(x4)
    x5 = Add()([x4, x_start])
    out = Conv2D(1, kernel_size=3, strides=1, padding='same',
                 dtype='float64')(x5)
    return tf.keras.Model(inputs=lr_image, outputs=out, name=name)


def cab(input):
    x = GlobalAveragePooling2D()(input)
    x = Permute((2, 1))(x)  # Act on channel axis
    x = Conv1D()(x)
    x = ReLU()(x)
    x = Conv1D()(x)
    x = Activation('sigma')(x)
    x = Permute((2, 1))(x)  # Restore original channel axis
    out = Multiply()([x, input])
    return out

In [4]:
Model = dcr_net()

In [5]:
csv_logger = CSVLogger('C:\\Users\\Furkan\\Desktop\\BLG4901E\\CNN\\log.csv', append=True, separator=';')

In [6]:
checkpoint_filepath = 'C:\\Users\\Furkan\\Desktop\\BLG4901E\\CNN\\chkpt\\model_checkpoint.h5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='loss',
    mode='max',
    save_best_only=False,
    save_weights_only=False,
    save_freq="epoch")

In [7]:
Model = tf.keras.models.load_model(checkpoint_filepath) # ÇOK DİKKAT ET BU KAPALI BUNUN KAPALI OLMASI GERKEN DURUMLAR OLABİLİR 

In [8]:
optimizer = optimizers.Adam(learning_rate=1e-4)
Model.compile(optimizer=optimizer, loss='mse', metrics=["mse", "mae"])
Model.summary()

Model: "DCRN_model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, None, None,  0           []                               
                                 1)]                                                              
                                                                                                  
 conv2d (Conv2D)                (None, None, None,   160         ['input[0][0]']                  
                                16)                                                               
                                                                                                  
 re_lu (ReLU)                   (None, None, None,   0           ['conv2d[0][0]']                 
                                16)                                                      

In [10]:
# epochs = 460 (0.8 test size)
st_ix = 0
epochs = 10
file_path = 'C:\\Users\\Furkan\\Desktop\\BLG4901E\\CNN\\devam_indisi.txt'
kac_epoch = 'C:\\Users\\Furkan\\Desktop\\BLG4901E\\CNN\\d_kac_epoch.txt'

for i in range(3):
    while True:
        with open(file_path, 'r') as file:
            st_ix = int(file.read())
        
        if st_ix >= 464:
            with open(file_path, 'w') as file:
                file.write("0")
            with open(kac_epoch, 'r') as file:
                a = file.read()
                a = int(a)
            with open(kac_epoch, 'w') as file:
                a += 1
                file.write(f"{a}")
            break

        X_train, y_train, st_ix = data_generator(st_ix, 8)

        Model.fit(
            x=X_train,
            y=y_train,
            epochs=1,
            batch_size=4,
            shuffle=True,
            callbacks=[model_checkpoint_callback]
        )

        with open(file_path, 'w') as file:
            file.write(f"{st_ix}")

        gc.collect()


262/262 [==============================] - 52s 197ms/step - loss: 0.0011 - mse: 0.0011 - mae: 0.0167


TypeError: can only concatenate str (not "int") to str

In [11]:
gc.collect()
with open('C:\\Users\\Furkan\\Desktop\\BLG4901E\\CNN\\devam_indisi.txt', 'r') as file:
    st_ix = int(file.read())

In [ ]:
X_val.shape

In [12]:
X_val, y_val, _ = data_generator(start_index=464,batch_size=50)

pred_loss = 0
x_loss = 0
pred_batch = 100
last = 1000
for i in range(0, last, pred_batch):
    predictions = Model.predict(X_val[i:i + pred_batch])
    predictions[predictions < 0.0] = 0.0
    predictions[predictions > 1.0] = 1.0
    pred_loss += np.sum(np.square(predictions[0:pred_batch] - y_val[i:i + pred_batch])) 
    x_loss += np.sum(np.square(X_val[i:i + pred_batch] - y_val[i:i + pred_batch])) 

pred_loss = pred_loss / last
x_loss = x_loss / last



4/4 [==============================] - 1s 319ms/step


In [13]:
print(f"pred loss: {pred_loss}")
print(f"x_loss: {x_loss}")

pred loss: 86.94172260212787
x_loss: 90.16467984621299


In [ ]:
idx = 0
cv2.imwrite("C:\\Users\\Furkan\\Desktop\\BLG4901E\\CNN\\gorseller\\X.png", X_val[99] * 255.0)
cv2.imwrite("C:\\Users\\Furkan\\Desktop\\BLG4901E\\CNN\\gorseller\\pred.png", predictions[9] * 255.0)
cv2.imwrite("C:\\Users\\Furkan\\Desktop\\BLG4901E\\CNN\\gorseller\\y.png", y_val[99] * 255.0)